<a href="https://colab.research.google.com/github/dowrave/Data_Analysis_Projects/blob/main/Dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from pandas import CategoricalDtype
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')

In [ ]:
train.head(2)

In [ ]:
train.info()

# 데이터 탐색

### 질적 데이터

- 전체 데이터 중 유망주의 비율 : $0.361$

In [ ]:
train['Prospect'].sum() / train['Prospect'].count()

#### 1. `PreferredFoot, WorkRate`
선요약 )  
- 특히 `WorkRate`의 경우는 높을수록 좋을 것이라는 생각을 하고 접근했지만, 가장 좋은 수치는 `Medium, Medium`에서 나왔음
- `PreferredFoot`의 경우 또한 `Left`에서 희소성이 있을 거라 생각하고 접근했지만, 그렇지는 않았음
- 둘 다 특별히 고칠 요소는 없는 것으로 보임


In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))

fig.suptitle("Preferred Foot : Total vs Prospect")
(train['PreferredFoot'].value_counts()
                        .to_frame()
                        .T
                        .plot(kind = 'bar', 
                              stacked = True, 
                              rot = 0,
                              ax = ax[0])
                        )
ax[0].legend().set_visible(False)

(train[train['Prospect'] == 1]['PreferredFoot'].value_counts()
                                                .to_frame()
                                                .T
                                                .plot(kind = 'bar', 
                                                      stacked = True, 
                                                      rot = 0,
                                                      ax = ax[1],
))

ax[1].set_ylim(ax[0].get_ylim())

In [ ]:
# 각 발 전체 인원 중 유망주 비율
print(train[(train['PreferredFoot'] == 'Left')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['PreferredFoot'] == 'Left']['ID'].count(),
      train[(train['PreferredFoot'] == 'Right')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['PreferredFoot'] == 'Right']['ID'].count(),
)
# 전체 인원 중 각 발의 비율
print(train[train['PreferredFoot'] == 'Left']['ID'].count() / train['ID'].count(),
      train[train['PreferredFoot'] == 'Right']['ID'].count() / train['ID'].count(),
)
# 유망주 전체 인원 중 각 발의 비율
print(train[(train['PreferredFoot'] == 'Left')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['Prospect'] == 1]['ID'].count(),
      train[(train['PreferredFoot'] == 'Right')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['Prospect'] == 1]['ID'].count(),
)

1. 각 발에 대해 전체 인원 중 유망주 비율
- 왼발 : $0.33$
- 오른발 : $0.37$

2. 전체 인원과 각 발의 비율
- 왼발 : $0.24$
- 오른발 : $0.75$

3. 유망주 전체 인원과 각 발의 비율
- 왼발 : $0.22$
- 오른발 : $0.78$

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (10, 10))

for i, val in enumerate(['AttackingWorkRate', 'DefensiveWorkRate']):
  if i == 0:
    color = np.array(sns.color_palette('OrRd'))[[1, 3, 5], :]
  elif i == 1:
    color = np.array(sns.color_palette('Blues'))[[1, 3, 5], :]
  
  (train[val].value_counts()[['Low', 'Medium', 'High']]
                          .to_frame()
                          .T
                          .plot(kind = 'bar', 
                                stacked = True, 
                                rot = 0,
                                ax = ax[i][0],
                                color = color)
                          )
  ax[i][0].legend().set_visible(False)

  (train[train['Prospect'] == 1][val].value_counts()[['Low', 'Medium', 'High']]
                                                  .to_frame()
                                                  .T
                                                  .plot(kind = 'bar', 
                                                        stacked = True, 
                                                        rot = 0,
                                                        ax = ax[i][1],
                                                        color = color
  ))

  ax[i][1].set_ylim(ax[i][0].get_ylim())

In [ ]:
# 위 사항은 수치적으로 보고 싶음 : WorkRate에는 총 9가지 유형이 있는데, 이에 따른 생존율을 비교해보자
temp = (train[['ID', 'AttackingWorkRate', 'DefensiveWorkRate', 'Prospect']]
                                                              .groupby(['AttackingWorkRate', 'DefensiveWorkRate', 'Prospect'])['ID']
                                                              .count()
                                                              .to_frame()
                                                              .reset_index().rename(columns = {"ID" : 'counts'})
)
temp.head(2)

In [ ]:
sns.catplot(data = temp, 
            x = 'AttackingWorkRate', 
            y = 'counts', 
            hue = 'DefensiveWorkRate', 
            col = 'Prospect', 
            kind = 'bar', 
            order = ['Low', 'Medium', 'High'], 
            hue_order = ['Low', 'Medium', 'High'],
            ax = ax)
plt.suptitle("ATT / DEF WorkRate and The Num of Prospect", y = 1.05, fontsize = 15)
plt.tight_layout()

In [ ]:
temp['rates'] = temp['counts'] / temp.groupby(['AttackingWorkRate', 'DefensiveWorkRate'])['counts'].transform("sum")
temp_hm = temp[temp['Prospect'] == 1][:]
temp_hm = temp_hm.pivot(index = 'AttackingWorkRate', columns = 'DefensiveWorkRate', values = 'rates').loc[['High', 'Medium', 'Low'], ['Low', 'Medium', 'High']]

In [ ]:
plt.figure(figsize = (7, 7))
sns.heatmap(data = temp_hm, annot = True, fmt = '.3f', cmap = 'Blues', vmin = 0, vmax = 1)
plt.title("Att / Def WorkRate and Prospect Rate")

- `Low, Low`에 있는 요소는 샘플이 1개니까 크게 고려할 요소는 아님
- 애초에 예상한 건 `오른쪽 위로 갈수록 활동량이 좋으니 Prospect 비율이 높을 것이다` 였는데, 그렇지는 않은 것으로 나타남

#### 2. `Position`

In [ ]:
# 먼저 시각화 하고 시작함
fig, ax = plt.subplots(figsize = (10, 5))
position_order = ['GK', 'LB', 'CB', 'RB', 'LWB', 'CDM', 'RWB', 'LM', 'CM', 'RM', 'CAM', 'LW', 'CF', 'RW', 'ST']
(train[['Position', 'Prospect']].value_counts()
                                .to_frame()
                                .reset_index()
                                .rename(columns = {0 : 'counts'})
                                .pivot(index = 'Position', 
                                       columns = 'Prospect', 
                                       values = 'counts')
                                .loc[position_order]
                                .plot(kind = 'bar', stacked = True, ax = ax)
)
plt.title("Position and Prospect")
plt.gca().set_axisbelow(True)
plt.grid(True, axis = 'y')

- 이거를 막 어떻게 다뤄봐야겠다는 생각은 딱 드는 게 없음
- 그래서 시각화만 해두고 넘어감

### 양적 데이터
1. 

In [ ]:
plt.figure(figsize = (7, 7))

corr = train.corr()
mask = np.zeros_like(corr, dtype = bool)
mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것

sns.heatmap(data = corr, cmap = 'coolwarm', mask = mask) 

- 위 히트맵에서, 일부 필요없는 부분을 제외하고 시각화 함
- 보고 싶은 건 2가지임
  1. 각종 스탯 간의 관계
  2. 각종 스탯과 `Position Rating`의 관계
- 또한, 음의 상관 관계는 크게 중요한 부분이 아니라고 생각되어 제외하겠음

In [ ]:
border = train.corr().columns.get_loc('GKReflexes') + 1 # 47

In [ ]:
np.where(np.all(corr1 > 0.5))

In [ ]:
threshold = np.where(corr1 > 0.5, 0, 1)

In [ ]:
# 1. 스탯 간의 관계 보기

# 근데 그냥 0 이상으로 두는게 낫지 않을까 싶으요?
# 요건 솔직히 어떻게 해놔야 될지 모르겠다. 다음에 보도록 하자.
corr1 = train.corr().iloc[:border, :border]
mask = np.zeros_like(corr1, dtype = bool)
mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것


plt.figure(figsize = (12, 12))
sns.heatmap(data = corr1, 
            cmap = 'Reds', 
            mask = mask | np.where(corr1 > 0.5, 0, 1), 
            vmin = 0, 
            vmax = 1) 

In [ ]:
corr2 = train.corr().iloc[:border, border:]
# mask = np.zeros_like(corr2, dtype = bool)
# mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것

plt.figure(figsize = (12, 12))
sns.heatmap(data = corr2, 
            cmap = 'Reds', 
            mask = np.where(corr2 > 0.3, 0, 1),
            vmin = 0 , 
            vmax = 1) 

- 이렇게 놓고 봤을 때 명백히 나뉘는 구역이 3가지가 있음
  - `LM, CM, RM` 영역 및 이보다 공격적인 위치
  - `LWB, CDM, RWB` 영역 ~ 수비수 영역까지 수비적인 위치
  - `GK`

- 물론 그 속에서도 중앙이냐 측면이냐에 따른 역할 차이가 보이기는 함
  - 특히 중앙 미드필더부터 수비적 포지션에 대해서 그 차이가 더 두드러져 보임

# 전처리
- 나중에 테스트 세트에도 적용해야 하니까 애껴둡시다.

In [ ]:
# 이거는 모델에 넣기 전에 진행하면 됨 - 먼저 나눠버리면 X축이 뭘 의미하는지 모르게 됨

foot_order = ['Left', 'Right']
work_rate_order = ['Low', 'Medium', 'High']
position_order = ['GK', 'LB', 'CB', 'RB', 'LWB', 'CDM', 'RWB', 'LM', 'CM', 'RM', 'CAM', 'LW', 'CF', 'RW', 'ST']

def column_label_order(df, col, order_lst):

  LE = LabelEncoder()
  LE.fit(df[col])
  LE.classes_ = np.array(order_lst)
  df[col] = LE.transform(df[col])


column_label_order(train, 'PreferredFoot', foot_order)
column_label_order(train, 'AttackingWorkRate', work_rate_order)
column_label_order(train, 'DefensiveWorkRate', work_rate_order)
column_label_order(train, 'Position', position_order)
